In [1]:
import numpy as np
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch as tc
from scipy.signal import fftconvolve
from torch import nn
from torch import optim
from torch.nn import functional as F
tc.random.manual_seed(0)

import numpy as np
from scipy.stats import multivariate_normal
from skimage.restoration import wiener
device = tc.device('cuda:0' if tc.cuda.is_available() else 'cpu')

# 準備するフィルタのサイズを5x5に固定
X, Y = np.meshgrid(np.arange(-2, 3, 1, dtype=np.float32), np.arange(-2, 3, 1, dtype=np.float32))
gauss_filters = []
std = 0.8
f = np.vectorize(
    lambda x, y: multivariate_normal([0.0, 0.0], np.diag([std]*2)).pdf([x, y])
)
kernel = f(X, Y)
kernel = kernel / kernel.sum()
kernel = kernel.astype(np.float32)
gauss1 = kernel

tc.random.manual_seed(0)
kernel = tc.rand((5, 5))
kernel = kernel / kernel.sum()
random1 = kernel

In [2]:
# idx: 0: gauss, 1: random
blur_kernels = [gauss1, random1]

class BluredMNIST(MNIST):
    def __getitem__(self, index):
        '''blur img and change target'''
        img, target = super().__getitem__(index)
        blur_index = tc.randint(0, 2, ())
        blur_kernel = blur_kernels[blur_index]
        img = fftconvolve(img[0], blur_kernel, mode='same')
        return img[np.newaxis, :], blur_index
        
transform = transforms.Compose([transforms.ToTensor()])
trainset = BluredMNIST(root='data/', train=True, download=True, transform=transform)
testset = BluredMNIST(root='data/', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, shuffle=True, batch_size=32)
testloader = DataLoader(testset, shuffle=False)

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 5, 5)
        self.fc1 = nn.Linear(5*4*4, 80)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(80, 40)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(40, 2)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 5*4*4)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [4]:
net = ConvNet()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(), lr=0.01)

In [5]:
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        lows, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(lows)
        outputs = tc.softmax(outputs, dim=1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('%d, %5d loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finish')

Finish


In [6]:
from sklearn.metrics import accuracy_score

y_true = []
y_pred = []
with tc.no_grad():
    for img, label in testloader:
        img, label = img.to(device), label.to(device)
        z = tc.argmax(net(img).squeeze(0))
        y_true.append(label.item())
        y_pred.append(z.item())

In [32]:
import numpy as np
y_true = np.array(y_true)
y_pred = np.array(y_pred)
accuracy_score(y_true, y_pred > 0)

0.4932

In [52]:
tc.save(net.state_dict(), '.models/feature_extraction.pth')